# Movie Recommender: Part 2: Collaborative Filtering

This Jupyter Notebook is part 2 of 3 to create a Recommender System using PySpark and the [MovieLens](https://grouplens.org/datasets/movielens/) dataset from GroupLens.   It uses the small dataset for education and development, which contains ~100,000 ratings from ~9,000 movies by ~600 users.  It was last updated September 2018 (as of 3/3/2022).  The ratings were created between March 29th, 1996 and September 24th, 2018.  More information can be found [here](https://files.grouplens.org/datasets/movielens/ml-latest-small-README.html).

We are interested in creating a recommender system that can accurately predict the ratings of movies for a given user.  We will be using collaborative-filtering first.

**Note**: The culmination of this project is a separate journal-formatted paper, so this Jupyter Notebook will have less text than usual.

**Notebook breakdown:**
- **Part 1:** Importing and EDA
- **Part 2:** Collaborative Filtering
- **Part 3:** Content-based Filtering

## Configuration:

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
INPUT_DIRECTORY = "/content/drive/MyDrive/GradSchool/DSCI632/MovieRecommender/data/" #for google mount
# INPUT_DIRECTORY = "./data/" #for jupyter notebook

In [4]:
%%capture 
#prevent large printout with %%capture

#Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#Install Apache Spark 3.2.1 with Hadoop 3.2, get zipped folder
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

#Unzip folder
!tar xvf spark-3.2.1-bin-hadoop3.2.tgz

#Install findspark, pyspark 3.2.1
!pip install -q findspark
!pip install pyspark==3.2.1

#Set variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.2.1-bin-hadoop3.2"

## Load Packages and Functions

In [5]:
from pyspark import SparkContext
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.sql import SparkSession

## Import Data

In [6]:
#create SparkSession and SparkContext objects
sc = SparkContext.getOrCreate()
spark = SparkSession.builder \
  .master("local[*]") \
  .config("spark.executor.memory", "70g") \
  .config("spark.driver.memory", "50g") \
  .config("spark.memory.offHeap.enabled",True) \
  .config("spark.memory.offHeap.size","16g") \
  .getOrCreate()

print('Master : ', sc.master)
print('Cores  : ', sc.defaultParallelism)

Master :  local[*]
Cores  :  2


In [7]:
spark.sparkContext._conf.getAll()

[('spark.driver.host', 'a047b0d518c6'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1646431384784'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir', 'file:/content/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.port', '33695'),
 ('spark.app.id', 'local-1646431386665')]

In [12]:
#Import data
file_path = INPUT_DIRECTORY + "ratings.csv"
ratings = spark.read.csv(file_path, header=True, inferSchema=True)
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



## ALS Model Creation

We'll split our data 80/20% into training/testing sets and set `seed` to 1 for reproducibility:

In [17]:
ratings = ratings.select("userId", "movieId", "rating")
(training_data, test_data) = ratings.randomSplit([.8, .2], seed=42)

Initialize our model.  We'll set the following parameters before optimizing hyperparameters:
- `nonnegative`: `True`. We only want non-negative numbers, as a negative rating has no meaning in this context.  
- `coldStartStrategy`: `"drop"`.  Helps avoid situations where all of a user's ratings are added to the training set only.  This data will not be used when calculating RMSE, because predictions on these users would be meaningless because there is nothing to test.
- `implicitPrefs`: `False`.  We have actual ratings, so we don't need to use implicit feedback.

In [16]:
from pyspark.ml.recommendation import ALS

als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
          nonnegative = True, coldStartStrategy = "drop", implicitPrefs = False)

Now we'll build our `ParamGridBuilder`:

In [31]:
from pyspark.ml.tuning import ParamGridBuilder

param_grid = ParamGridBuilder() \
                  .addGrid(als.rank, [5, 20]) \
                  .addGrid(als.maxIter, [5]) \
                  .addGrid(als.regParam, [0.01, 0.05, 1]) \
                  .build()

Next, we'll create our evaluator and use RMSE as our metric:

In [32]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  6


Create CrossValidator:

In [33]:
from pyspark.ml.tuning import CrossValidator

cv = CrossValidator(estimator = als, 
                    estimatorParamMaps= param_grid,
                    evaluator = evaluator,
                    numFolds = 5)

Fit Data:

In [34]:
model = cv.fit(training_data)

best_model = model.bestModel

Get information on the best model:

In [35]:
print(type(best_model))

print("\n**Best Model**")
print("  Rank:", best_model.rank)
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>

**Best Model**
  Rank: 5
  MaxIter: 5
  RegParam: 0.05


## Performance Evaluation

Let's generate predictions on the test data:

In [36]:
test_predictions = model.transform(test_data)
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   463|   1088|   3.5| 3.5744293|
|   580|   3175|   2.5| 3.4294555|
|   580|  44022|   3.5| 3.5521498|
|   362|   1645|   5.0| 3.7645638|
|   597|   1959|   4.0|  4.159972|
|   155|   3175|   4.0| 3.4803483|
|   368|   2122|   2.0|    2.6006|
|   115|   1645|   4.0| 3.4515243|
|   115|   3175|   4.0|  4.044546|
|    28|   1645|   2.5|  2.856431|
|    28|   3175|   1.5| 2.7337737|
|   587|   1580|   4.0|  4.012778|
|   332|   1645|   3.5|  3.097416|
|   332|   2366|   3.5|   3.51371|
|   577|   1580|   3.0| 3.3004088|
|   577|   1959|   4.0|  3.713063|
|   271|   6658|   2.0|  2.692251|
|   606|   1088|   3.0| 3.5075595|
|    91|   1580|   3.5|  3.105002|
|    91|   6620|   3.5| 3.4953837|
+------+-------+------+----------+
only showing top 20 rows



In [37]:
# Evaluate the "test_predictions" dataframe
RMSE = evaluator.evaluate(test_predictions)

# Print the RMSE
print(RMSE)

0.911022581949658


## Generate Recommendations:

In [39]:
# Generate top 10 movie recommendations for each user
userRecs = best_model.recommendForAllUsers(10)
userRecs.show(5, truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|userId|recommendations                                                                                                                                                                                                  |
+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1     |[{96004, 6.5534353}, {3379, 6.5534353}, {8477, 6.435445}, {58301, 6.318518}, {139640, 6.3116446}, {147410, 6.3116446}, {143031, 6.3116446}, {136341, 6.3116446}, {131610, 6.3116446}, {118270, 6.3116446}]       |
|3     |[{74754, 8.586728}, {5480, 7.422904}, {86320, 7.115943}, {121097, 7.1060853}, {128968, 6.767934}, {3837, 6.4242234},

In [177]:
userRecs_pandas = userRecs.toPandas()
userRecs_pandas.head()

,userId,recommendations
0,1,"[(96004, 6.553435325622559), (3379, 6.55343532..."
1,3,"[(74754, 8.5867280960083), (5480, 7.4229040145..."
2,5,"[(8477, 5.860734462738037), (3266, 5.842754364..."
3,6,"[(136341, 7.057549476623535), (109241, 7.05754..."
4,9,"[(84847, 7.28206205368042), (3223, 7.243201255..."


In [171]:
def get_movie_title_from_id(movieId):
  title =  movie_titles.loc[movie_titles["movieId"]==movieId,"title"].item()
  return title

In [195]:
def get_user_recommended_movies(recs_df, userId):
  try:
    recommendations = recs_df[recs_df["userId"] == userId]["recommendations"]
    for movie in recommendations[0]:
      print(f"Movie: \n{get_movie_title_from_id(movie[0])}\nPredicted Rating: {movie[1]}\n")
  except:
    print("That userId does not exist in the dataset.  Try another.")

In [196]:
import pandas as pd

file_path = INPUT_DIRECTORY + "movies.csv"
movie_titles = pd.read_csv(file_path)
movie_titles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [197]:
get_movie_title_from_id(10)

'GoldenEye (1995)'

In [198]:
get_user_recommended_movies(userRecs_pandas, 1)

Movie: 
Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993)
Predicted Rating: 6.553435325622559

Movie: 
On the Beach (1959)
Predicted Rating: 6.553435325622559

Movie: 
Jetée, La (1962)
Predicted Rating: 6.4354448318481445

Movie: 
Funny Games U.S. (2007)
Predicted Rating: 6.318518161773682

Movie: 
Ooops! Noah is Gone... (2015)
Predicted Rating: 6.311644554138184

Movie: 
A Perfect Day (2015)
Predicted Rating: 6.311644554138184

Movie: 
Jump In! (2007)
Predicted Rating: 6.311644554138184

Movie: 
Scooby-Doo! and the Samurai Sword (2009)
Predicted Rating: 6.311644554138184

Movie: 
Willy/Milly (1986)
Predicted Rating: 6.311644554138184

Movie: 
Hellbenders (2012)
Predicted Rating: 6.311644554138184



In [199]:
#try a user that doesn't exist
get_user_recommended_movies(userRecs_pandas, 2)

That userId does not exist in the dataset.  Try another.
